# Fake News Classifier Using LSTM

Dataset: https://www.kaggle.com/c/fake-news/data#

In [1]:
import pandas as pd

In [2]:
## Load the Dataset
df = pd.read_csv('train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [3]:
df.shape

(20800, 5)

In [4]:
## Drop the Null value
df = df.dropna()

In [5]:
df.shape

(18285, 5)

In [6]:
## Get the Independent Features
X = df.drop(columns='label', axis=1)
X.shape

(18285, 4)

In [7]:
## Get the Dependent Feature
y = df['label']
y.shape

(18285,)

Import dependencies for LSTM model

In [42]:
from tensorflow.keras.layers import Embedding, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [9]:
## Vocabulary size
voc_size = 5000

In [10]:
messages = X['title']

In [11]:
len(messages)

18285

In [12]:
messages[1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

Text Preprocessing

In [13]:
import nltk
import re
from nltk.corpus import stopwords

In [14]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [15]:
## Dataset Preprocessing
from nltk.stem.porter import PorterStemmer ##stemming purpose
ps = PorterStemmer()
corpus = []
for i in messages:
    review = re.sub('[^a-zA-Z]', ' ',i).lower().split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [18]:
corpus[:10]

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag']

ONE HOT ENCODING

In [21]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr[:5]

[[4024, 1040, 2417, 2321, 3664, 3471, 3748, 1249, 3173, 1098],
 [4001, 4552, 3054, 3291, 2364, 4059, 1377],
 [1369, 4516, 4373, 3346],
 [1271, 531, 1461, 480, 257, 219],
 [4865, 2364, 3186, 4102, 2230, 3362, 2364, 1071, 4650, 595]]

Padding for make length of each sentences are same

In [22]:
sent_length=15
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 1249 3173 1098]
 [   0    0    0 ... 2364 4059 1377]
 [   0    0    0 ... 4516 4373 3346]
 ...
 [   0    0    0 ... 2953 2186 1423]
 [   0    0    0 ... 3199 2988 1078]
 [   0    0    0 ... 1248 4686  767]]


In [23]:
embedded_docs.shape

(18285, 15)

Creating model

In [34]:
features=40  ##features representation
model=Sequential()

## Add Input/Embedding Layer
model.add(Embedding(voc_size,features,input_length=sent_length))

## Add Hidden/LSTM Layer
model.add(LSTM(100))  ## Taking 1 LSTM Layer with 100 Neurons

## Add Output Layer
model.add(Dense(1,activation='sigmoid'))

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 15, 40)            200000    
                                                                 
 lstm_3 (LSTM)               (None, 100)               56400     
                                                                 
 dense_3 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [35]:
## Convert into Array
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [36]:
X_final.shape,y_final.shape

((18285, 15), (18285,))

Splitng into train and test data

In [37]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

Model Training

In [39]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=100)

Epoch 1/10
123/123 [==============================] - 8s 40ms/step - loss: 0.0024 - accuracy: 0.9994 - val_loss: 0.6508 - val_accuracy: 0.9074
Epoch 2/10
123/123 [==============================] - 5s 38ms/step - loss: 7.2847e-04 - accuracy: 0.9998 - val_loss: 0.6908 - val_accuracy: 0.9080
Epoch 3/10
123/123 [==============================] - 5s 40ms/step - loss: 3.3146e-04 - accuracy: 0.9999 - val_loss: 0.7311 - val_accuracy: 0.9077
Epoch 4/10
123/123 [==============================] - 5s 39ms/step - loss: 1.8674e-04 - accuracy: 0.9999 - val_loss: 0.7638 - val_accuracy: 0.9087
Epoch 5/10
123/123 [==============================] - 5s 41ms/step - loss: 1.7419e-04 - accuracy: 1.0000 - val_loss: 0.7613 - val_accuracy: 0.9070
Epoch 6/10
123/123 [==============================] - 5s 41ms/step - loss: 1.2712e-04 - accuracy: 1.0000 - val_loss: 0.7728 - val_accuracy: 0.9084
Epoch 7/10
123/123 [==============================] - 5s 39ms/step - loss: 9.5960e-05 - accuracy: 1.0000 - val_loss: 0.787

# NOTICE PROBLEM

Model Training Accuracy is 100% & Testing accurace is 90%. So, it is overfitting situation.
Now, We need to regulized model with add droput layer.

Regulized the model 

In [73]:
features=30  ##features representation
model=Sequential()

## Add Input/Embedding Layer
model.add(Embedding(voc_size,features,input_length=sent_length))
model.add(Dropout(0.7))

## Add Hidden/LSTM Layer
model.add(LSTM(100))  ## Taking 1 LSTM Layer with 100 Neurons
model.add(Dropout(0.7))

## Add Output Layer
model.add(Dense(1,activation='sigmoid'))

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_11 (Embedding)    (None, 15, 30)            150000    
                                                                 
 dropout_10 (Dropout)        (None, 15, 30)            0         
                                                                 
 lstm_9 (LSTM)               (None, 100)               52400     
                                                                 
 dropout_11 (Dropout)        (None, 100)               0         
                                                                 
 dense_9 (Dense)             (None, 1)                 101       
                                                                 
Total params: 202,501
Trainable params: 202,501
Non-trainable params: 0
_________________________________________________________________
None


In [74]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=100)

Epoch 1/10
123/123 [==============================] - 10s 47ms/step - loss: 0.4667 - accuracy: 0.7696 - val_loss: 0.2635 - val_accuracy: 0.8969
Epoch 2/10
123/123 [==============================] - 5s 39ms/step - loss: 0.2167 - accuracy: 0.9132 - val_loss: 0.2032 - val_accuracy: 0.9157
Epoch 3/10
123/123 [==============================] - 5s 39ms/step - loss: 0.1663 - accuracy: 0.9332 - val_loss: 0.1894 - val_accuracy: 0.9226
Epoch 4/10
123/123 [==============================] - 5s 40ms/step - loss: 0.1403 - accuracy: 0.9435 - val_loss: 0.1977 - val_accuracy: 0.9223
Epoch 5/10
123/123 [==============================] - 5s 42ms/step - loss: 0.1220 - accuracy: 0.9537 - val_loss: 0.1980 - val_accuracy: 0.9173
Epoch 6/10
123/123 [==============================] - 5s 43ms/step - loss: 0.1137 - accuracy: 0.9580 - val_loss: 0.2094 - val_accuracy: 0.9188
Epoch 7/10
123/123 [==============================] - 5s 44ms/step - loss: 0.1039 - accuracy: 0.9579 - val_loss: 0.2215 - val_accuracy: 0.919

Performance Metrics And Accuracy

In [75]:
y_pred=model.predict(X_test)
y_pred=np.where(y_pred > 0.6, 1,0)
y_pred

189/189 [==============================] - 2s 9ms/step


array([[1],
       [0],
       [0],
       ...,
       [0],
       [1],
       [1]])

In [76]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[3115,  304],
       [ 184, 2432]], dtype=int64)

In [77]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9191383595691798

In [78]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.94      0.91      0.93      3419
           1       0.89      0.93      0.91      2616

    accuracy                           0.92      6035
   macro avg       0.92      0.92      0.92      6035
weighted avg       0.92      0.92      0.92      6035

